In [1]:
from TriggerStudyBinaries_v5 import *

example_file = np.loadtxt("/cr/tempdata01/filip/iRODS/Background/jaco_randoms0000.csv")
example_trace = example_file[0]

2022-08-18 16:00:43.275094: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-18 16:00:43.275118: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
plt.rcParams['figure.figsize'] = [20, 10]
plt.plot(range(len(example_trace) - 1), example_trace[1:] + example_trace[0])
plt.axhline(example_trace[0], ls = "--", c = "k", lw = 2)
plt.axhline(np.mean(example_trace[1:] + example_trace[0]), ls = "--", c = "orange", lw = 2)

In [27]:
class ElectronicsError(Exception) : pass
class SignalError(Exception) : pass

# Implementation of Tobias Schulz algorithm in python
def calculate_baseline(*args) -> list :

    baselines = []
    for window in args:

        while True:

            mode, count = None, 0
            
            # calculate mode of the trace windows
            for value in range(int(min(window)), int(max(window))):
                current_count = window.count(value)
                if current_count > count:
                    mode, count = value, current_count

            upper, lower = mode + 2* np.std(window), mode - 2* np.std(window)
            to_remove = [bin for bin in window if bin > upper or bin < lower]           
            if to_remove != []: [window.remove(value) for value in to_remove]
            else: break

        baselines.append(np.mean(window))

    return baselines

def correct_trace(trace : np.ndarray) -> int :

    i, f = calculate_baseline(list(trace[:300]), list(trace[-300:]))

    if i - f > 0.1: raise SignalError
    elif i - f < -0.5: raise ElectronicsError
    else:

        baseline = calculate_baseline(list(trace))[0]
            



In [28]:
correct_trace(example_trace)

0.875
